In [37]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow import keras
from tensorflow.keras import layers

# Load EEG data from CSV
eeg_data = pd.read_csv(r"D:\Joel\Bioamp_Data\Data\Jaw_Clench\2_mins_Alpha\2_min_Alpha_Input.csv")

# Define the number of rows in each epoch (4 seconds)
epoch_length = 25000

# Initialize lists to store epochs and labels
epochs = []
labels = []

# Extract EEG data and labels
for i in range(0, len(eeg_data), epoch_length):
    epoch = eeg_data.iloc[i:i + epoch_length]
    if len(epoch) == epoch_length:
        eeg_epoch = epoch.iloc[:, 0].values  # Assuming EEG data is in the first column
        label = epoch.iloc[0, 1]  # Assuming the target is in the second column
        epochs.append(eeg_epoch)
        labels.append(label)

# Convert to NumPy arrays
epochs = np.array(epochs)
labels = np.array(labels)

# Perform label encoding for binary classification (REST, JAW)
label_encoder = LabelEncoder()
labels_encoded = label_encoder.fit_transform(labels)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(epochs, labels_encoded, test_size=0.3, random_state=42)

# Reshape data to match the input shape
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

# Define EEGNet architecture
model = keras.Sequential()
model.add(layers.Conv1D(8, kernel_size=64, input_shape=(X_train.shape[1], 1), padding='same'))
model.add(layers.BatchNormalization())
model.add(layers.DepthwiseConv1D(kernel_size=64, depth_multiplier=2, depthwise_initializer='he_normal', use_bias=False, padding='valid'))
model.add(layers.BatchNormalization())
model.add(layers.DepthwiseConv1D(kernel_size=16, depth_multiplier=2, depthwise_initializer='he_normal', use_bias=False, padding='same'))
model.add(layers.BatchNormalization())
model.add(layers.Activation('elu'))
model.add(layers.AveragePooling1D(pool_size=4))
model.add(layers.Dropout(0.5))
model.add(layers.SeparableConv1D(16, kernel_size=16, activation='elu', depthwise_initializer='he_normal', use_bias=False, padding='same'))
model.add(layers.BatchNormalization())
model.add(layers.SeparableConv1D(16, kernel_size=16, activation='elu', depthwise_initializer='he_normal', use_bias=False, padding='same'))
model.add(layers.BatchNormalization())
model.add(layers.AveragePooling1D(pool_size=4))
model.add(layers.Dropout(0.5))
model.add(layers.Flatten())
model.add(layers.Dense(1, activation='sigmoid'))  # Binary classification

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=40, batch_size=32, validation_split=0.1)

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Loss: {loss}, Test Accuracy: {accuracy}")


Epoch 1/40
7/7 [==============================] - 15s 2s/step - loss: nan - accuracy: 0.2336 - val_loss: nan - val_accuracy: 0.2083
Epoch 2/40
7/7 [==============================] - 11s 2s/step - loss: nan - accuracy: 0.2336 - val_loss: nan - val_accuracy: 0.2083
Epoch 3/40
7/7 [==============================] - 11s 2s/step - loss: nan - accuracy: 0.2336 - val_loss: nan - val_accuracy: 0.2083
Epoch 4/40
7/7 [==============================] - 12s 2s/step - loss: nan - accuracy: 0.2336 - val_loss: nan - val_accuracy: 0.2083
Epoch 5/40
7/7 [==============================] - 12s 2s/step - loss: nan - accuracy: 0.2336 - val_loss: nan - val_accuracy: 0.2083
Epoch 6/40
7/7 [==============================] - 12s 2s/step - loss: nan - accuracy: 0.2336 - val_loss: nan - val_accuracy: 0.2083
Epoch 7/40
7/7 [==============================] - 12s 2s/step - loss: nan - accuracy: 0.2336 - val_loss: nan - val_accuracy: 0.2083
Epoch 8/40
7/7 [==============================] - 12s 2s/step - loss: nan - 

KeyboardInterrupt: 

In [32]:
import pandas as pd

# Read the CSV file into a DataFrame
file_path = r"C:\Users\HP\Downloads\George_Blink_Rest_2mins.csv"  # Replace with the path to your CSV file
df = pd.read_csv(file_path,nrows=1200000)

# Add a third column named 'NewColumn' and fill it based on the specified pattern
pattern = ['REST', 'BLINK']
rows_per_pattern = 10000

df['TARGET'] = [pattern[i // rows_per_pattern % len(pattern)] for i in range(len(df))]

# Save the updated DataFrame to a new CSV file
output_file_path = r"D:\Joel\Bioamp_Data\Data\Jaw_Clench\George_Blink_Rest_2mins.csv"  # Replace with your desired output file path
df.to_csv(output_file_path, index=False)

print(f'Data has been processed and saved to {output_file_path}')


Data has been processed and saved to D:\Joel\Bioamp_Data\Data\Jaw_Clench\George_Blink_Rest_2mins.csv


In [1]:
import os
import pandas as pd

# Replace the directory path with the path to your CSV files
directory_path = r'D:\Joel\Bioamp_Data\Data\Jaw_Clench\2_mins_Alpha\JAW'

# List all CSV files in the directory
csv_files = [file for file in os.listdir(directory_path) if file.endswith('.csv')]

# Initialize an empty DataFrame to store the merged data
merged_df = pd.DataFrame()

# Loop through each CSV file and append its content to the merged DataFrame
for file in csv_files:
    file_path = os.path.join(directory_path, file)
    df = pd.read_csv(file_path)
    merged_df = merged_df.append(df, ignore_index=True)

# Specify the path for the output merged CSV file
output_file_path = r'D:\Joel\Bioamp_Data\Data\Jaw_Clench\2_mins_Alpha\JAW\Joel_2min_alpha.csv'

# Save the merged DataFrame to a new CSV file
merged_df.to_csv(output_file_path, index=False)

print(f'Data from {len(csv_files)} CSV files has been merged and saved to {output_file_path}')


C:\Users\HP\AppData\Local\Temp\ipykernel_20812\2733951060.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  merged_df = merged_df.append(df, ignore_index=True)
C:\Users\HP\AppData\Local\Temp\ipykernel_20812\2733951060.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  merged_df = merged_df.append(df, ignore_index=True)


Data from 2 CSV files has been merged and saved to D:\Joel\Bioamp_Data\Data\Jaw_Clench\2_mins_Alpha\JAW\Joel_2min_alpha.csv


In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow import keras
from tensorflow.keras import layers

# Load EEG data from CSV
eeg_data = pd.read_csv(r"D:\Joel\Bioamp_Data\Data\Jaw_Clench\Sandra_Jaw_Target_Input.csv")

# Define the number of rows in each epoch (4 seconds)
epoch_length = 12500

# Initialize lists to store epochs and labels
epochs = []
labels = []

# Extract EEG data and labels
for i in range(0, len(eeg_data), epoch_length):
    epoch = eeg_data.iloc[i:i + epoch_length]
    if len(epoch) == epoch_length:
        eeg_epoch = epoch.iloc[:, 0].values  # Assuming EEG data is in the first column
        label = epoch.iloc[0, 1]  # Assuming the target is in the second column
        epochs.append(eeg_epoch)
        labels.append(label)

# Convert to NumPy arrays
epochs = np.array(epochs)
labels = np.array(labels)

# Perform label encoding for binary classification (REST, JAW)
label_encoder = LabelEncoder()
labels_encoded = label_encoder.fit_transform(labels)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(epochs, labels_encoded, test_size=0.3, random_state=42)

# Reshape data to match the input shape
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

# Define EEGNet architecture
model = keras.Sequential()
model.add(layers.Conv1D(8, kernel_size=64, input_shape=(X_train.shape[1], 1), padding='same'))
model.add(layers.BatchNormalization())
model.add(layers.DepthwiseConv1D(kernel_size=64, depth_multiplier=2, depthwise_initializer='he_normal', use_bias=False, padding='valid'))
model.add(layers.BatchNormalization())
model.add(layers.DepthwiseConv1D(kernel_size=16, depth_multiplier=2, depthwise_initializer='he_normal', use_bias=False, padding='same'))
model.add(layers.BatchNormalization())
model.add(layers.Activation('elu'))
model.add(layers.AveragePooling1D(pool_size=4))
model.add(layers.Dropout(0.5))
model.add(layers.SeparableConv1D(16, kernel_size=16, activation='elu', depthwise_initializer='he_normal', use_bias=False, padding='same'))
model.add(layers.BatchNormalization())
model.add(layers.SeparableConv1D(16, kernel_size=16, activation='elu', depthwise_initializer='he_normal', use_bias=False, padding='same'))
model.add(layers.BatchNormalization())
model.add(layers.AveragePooling1D(pool_size=4))
model.add(layers.Dropout(0.5))
model.add(layers.Flatten())
model.add(layers.Dense(1, activation='sigmoid'))  # Binary classification

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=5
          , batch_size=32, validation_split=0.2)

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Loss: {loss}, Test Accuracy: {accuracy}")


Epoch 1/5
1/1 [==============================] - 3s 3s/step - loss: 0.8034 - accuracy: 0.5833 - val_loss: 0.6414 - val_accuracy: 0.7500
Epoch 2/5
1/1 [==============================] - 0s 278ms/step - loss: 0.5178 - accuracy: 0.6667 - val_loss: 0.6320 - val_accuracy: 1.0000
Epoch 3/5
1/1 [==============================] - 0s 266ms/step - loss: 0.2003 - accuracy: 0.9167 - val_loss: 0.6253 - val_accuracy: 1.0000
Epoch 4/5
1/1 [==============================] - 0s 265ms/step - loss: 0.1617 - accuracy: 1.0000 - val_loss: 0.6241 - val_accuracy: 1.0000
Epoch 5/5
1/1 [==============================] - 0s 53ms/step - loss: 0.6664 - accuracy: 0.7143
Test Loss: 0.6663824915885925, Test Accuracy: 0.7142857313156128


# SAMPLE RNN MODEL 

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

# Load EEG data from CSV
eeg_data = pd.read_csv(r"D:\Joel\Bioamp_Data\Data\Jaw_Clench\Sandra_Jaw_Target_Input.csv")

# Define the number of rows in each epoch (4 seconds)
epoch_length = 12500

# Initialize lists to store epochs and labels
epochs = []
labels = []

# Extract EEG data and labels
for i in range(0, len(eeg_data), epoch_length):
    epoch = eeg_data.iloc[i:i + epoch_length]
    if len(epoch) == epoch_length:
        eeg_epoch = epoch.iloc[:, 0].values  # Assuming EEG data is in the first column
        label = epoch.iloc[0, 1]  # Assuming the target is in the second column
        epochs.append(eeg_epoch)
        labels.append(label)

# Convert to NumPy arrays
epochs = np.array(epochs)
labels = np.array(labels)

# Perform label encoding for binary classification (REST, JAW)
label_encoder = LabelEncoder()
labels_encoded = label_encoder.fit_transform(labels)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(epochs, labels_encoded, test_size=0.3, random_state=42)

# Reshape data to match the input shape
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

# Build the LSTM model
model = Sequential()
model.add(LSTM(units=50, activation='relu', input_shape=(X_train.shape[1], 1)))
model.add(Dense(units=1, activation='sigmoid'))  # Assuming binary classification, adjust for multi-class

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Accuracy: {accuracy * 100:.2f}%')


Epoch 1/10
1/1 [==============================] - 7s 7s/step - loss: nan - accuracy: 0.6250 - val_loss: nan - val_accuracy: 0.1429
Epoch 2/10
1/1 [==============================] - 4s 4s/step - loss: nan - accuracy: 0.6250 - val_loss: nan - val_accuracy: 0.1429
Epoch 3/10
1/1 [==============================] - 4s 4s/step - loss: nan - accuracy: 0.6250 - val_loss: nan - val_accuracy: 0.1429
Epoch 4/10
1/1 [==============================] - 4s 4s/step - loss: nan - accuracy: 0.6250 - val_loss: nan - val_accuracy: 0.1429
Epoch 5/10
1/1 [==============================] - 5s 5s/step - loss: nan - accuracy: 0.6250 - val_loss: nan - val_accuracy: 0.1429
Epoch 6/10
1/1 [==============================] - 4s 4s/step - loss: nan - accuracy: 0.6250 - val_loss: nan - val_accuracy: 0.1429
Epoch 7/10
1/1 [==============================] - 5s 5s/step - loss: nan - accuracy: 0.6250 - val_loss: nan - val_accuracy: 0.1429
Epoch 8/10
1/1 [==============================] - 4s 4s/step - loss: nan - accuracy

# 1d Convo NET

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense

# Load EEG data from CSV without headers
eeg_data = pd.read_csv(r"D:\Joel\Bioamp_Data\Data\Jaw_Clench\Sandra_Jaw_Target_Input.csv", header=None)

# Define the number of rows in each second
rows_per_second = 5000

# Extract features (EEG signals) and labels
X = eeg_data.iloc[:, 0].values  # Assuming the first column contains data
y = eeg_data.iloc[:, 1].values  # Assuming the second column contains target labels

# Encode target labels
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

# Normalize EEG data
scaler = StandardScaler()
X = scaler.fit_transform(X.reshape(-1, 1))  # Reshape for normalization

# Reshape data into 3D array (samples, time steps, features)
X_reshaped = X.reshape(-1, rows_per_second, 1)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_reshaped, y, test_size=0.2, random_state=42)

# Build the 1D CNN model
model = Sequential()
model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(rows_per_second, 1)))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(units=128, activation='relu'))
model.add(Dense(units=1, activation='sigmoid'))  # Adjust for multi-class

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Accuracy: {accuracy * 100:.2f}%')


ValueError: Found input variables with inconsistent numbers of samples: [60, 300000]

In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense

# Load data from CSV with no headers
df = pd.read_csv(r"D:\Joel\Bioamp_Data\Data\Jaw_Clench\Sandra_Jaw_Target_Input.csv", header=None)
# Extract features (EEG signals) and labels
X = eeg_data.iloc[:, 0].values  # Assuming the first column contains data
y = eeg_data.iloc[:, 1].values 

# Encode target labels
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

# Assuming each data point is comma-separated values in a single string; adjust accordingly
X = np.array([np.fromstring(data, sep=',') for data in X])

# Normalize EEG data
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Reshape data for 1D CNN input (assuming 5000 time steps per sample)
X = X.reshape(-1, 5000, 1)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Build the 1D CNN model
model = Sequential()
model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(5000, 1)))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(units=128, activation='relu'))
model.add(Dense(units=1, activation='sigmoid'))  # Adjust for multi-class

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))


C:\Users\HP\AppData\Local\Temp\ipykernel_16752\2662027489.py:19: DeprecationWarning: string or file could not be read to its end due to unmatched data; this will raise a ValueError in the future.
  X = np.array([np.fromstring(data, sep=',') for data in X])


ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (300000,) + inhomogeneous part.

In [36]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense

# Load EEG data from CSV
eeg_data = pd.read_csv(r"D:\Joel\Bioamp_Data\Data\Jaw_Clench\2_mins_Alpha\2_min_Alpha_Input.csv")

# Define the number of rows in each epoch (4 seconds)
epoch_length = 25000

# Initialize lists to store epochs and labels
epochs = []
labels = []

# Extract EEG data and labels
for i in range(0, len(eeg_data), epoch_length):
    epoch = eeg_data.iloc[i:i + epoch_length]
    if len(epoch) == epoch_length:
        eeg_epoch = epoch.iloc[:, 0].values  # Assuming EEG data is in the first column
        label = epoch.iloc[0, 1]  # Assuming the target is in the second column
        epochs.append(eeg_epoch)
        labels.append(label)

# Convert to NumPy arrays
epochs = np.array(epochs)
labels = np.array(labels)

# Perform label encoding for binary classification (REST, JAW)
label_encoder = LabelEncoder()
labels_encoded = label_encoder.fit_transform(labels)

# Normalize EEG data
scaler = StandardScaler()
epochs = scaler.fit_transform(epochs)

# Reshape data to match the input shape
epochs = epochs.reshape(epochs.shape[0], epochs.shape[1], 1)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(epochs, labels_encoded, test_size=0.3, random_state=42)

# Build the 1D CNN model
model = Sequential()
model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(epoch_length, 1)))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(units=128, activation='relu'))
model.add(Dense(units=1, activation='sigmoid'))  # Adjust for multi-class

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=4, batch_size=32, validation_data=(X_test, y_test))
# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Accuracy: {accuracy * 100:.2f}%')


Epoch 1/4
8/8 [==============================] - 12s 1s/step - loss: nan - accuracy: 0.2353 - val_loss: nan - val_accuracy: 0.2621
Epoch 2/4
8/8 [==============================] - 11s 1s/step - loss: nan - accuracy: 0.2311 - val_loss: nan - val_accuracy: 0.2621
Epoch 3/4
8/8 [==============================] - 10s 1s/step - loss: nan - accuracy: 0.2311 - val_loss: nan - val_accuracy: 0.2621
Epoch 4/4
4/4 [==============================] - 1s 161ms/step - loss: nan - accuracy: 0.2621
Accuracy: 26.21%


In [35]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense

# Load EEG data from CSV
eeg_data = pd.read_csv(r"D:\Joel\Bioamp_Data\Data\Jaw_Clench\2_mins_Alpha\2_min_Alpha_Input.csv")

# Define the number of rows in each epoch (4 seconds)
epoch_length = 50000

# Initialize lists to store epochs and labels
epochs = []
labels = []

# Extract EEG data and labels
for i in range(0, len(eeg_data), epoch_length):
    epoch = eeg_data.iloc[i:i + epoch_length]
    if len(epoch) == epoch_length:
        eeg_epoch = epoch.iloc[:, 0].values  # Assum"D:\Joel\Bioamp_Data\Data\Jaw_Clench\Sandra_Jaw_Target_10000_Input.csv"ing EEG data is in the first column
        label = epoch.iloc[0, 1]  # Assuming the target is in the second column
        epochs.append(eeg_epoch)
        labels.append(label)

# Convert to NumPy arrays
epochs = np.array(epochs)
labels = np.array(labels)

# Perform label encoding for binary classification (REST, JAW)
label_encoder = LabelEncoder()
labels_encoded = label_encoder.fit_transform(labels)

# Normalize EEG data
scaler = StandardScaler()
epochs = scaler.fit_transform(epochs)

# Reshape data to match the input shape
epochs = epochs.reshape(epochs.shape[0], epochs.shape[1], 1)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(epochs, labels_encoded, test_size=0.3, random_state=42)

# Build the 1D CNN model
model = Sequential()
model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(epoch_length, 1)))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(units=128, activation='relu'))
model.add(Dense(units=1, activation='sigmoid'))  # Adjust for multi-class

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=3, batch_size=32 ,validation_data=(X_test, y_test))
# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Accuracy: {accuracy * 100:.2f}%')


Epoch 1/3
4/4 [==============================] - 11s 2s/step - loss: nan - accuracy: 0.2521 - val_loss: nan - val_accuracy: 0.1961
Epoch 2/3
4/4 [==============================] - 8s 2s/step - loss: nan - accuracy: 0.2521 - val_loss: nan - val_accuracy: 0.1961
Epoch 3/3
2/2 [==============================] - 0s 192ms/step - loss: nan - accuracy: 0.1961
Accuracy: 19.61%


In [8]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

# Load new data from CSV
new_data = pd.read_csv(r"D:\Joel\Bioamp_Data\Data\Jaw_Clench\Sandra_Jaw_Test.csv")  # Replace with the path to your new CSV file

# Extract features (assuming one column for data)
X_new = new_data.iloc[:, 0].values


# Normalize EEG data using the same scaler used during training


# Make predictions using the trained model
predictions = (X_new)

# Convert predictions to labels (e.g., binary classification)
predicted_labels = (predictions > 0.5).astype(int)

# If you performed label encoding during training, you can inverse transform the labels
predicted_labels_original = label_encoder.inverse_transform(predicted_labels)

# Print or use the predicted labels as needed
print(predicted_labels_original)


['REST' 'REST' 'REST' ... 'REST' 'REST' 'REST']


In [7]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder,MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPool1D,Dense, Flatten,BatchNormalization,LeakyReLU,Dropout,AveragePooling1D,GlobalAveragePooling1D

# Load EEG data from CSV
eeg_data = pd.read_csv(r"D:\Joel\Bioamp_Data\Data\Jaw_Clench\2_mins_Alpha\2_min_Alpha_Input.csv")

# Define the number of rows in each epoch (4 seconds)
epoch_length = 25000

# Initialize lists to store epochs and labels
epochs = []
labels = []

# Extract EEG data and labels
for i in range(0, len(eeg_data), epoch_length):
    epoch = eeg_data.iloc[i:i + epoch_length]
    if len(epoch) == epoch_length:
        eeg_epoch = epoch.iloc[:, 0].values  # Assum"D:\Joel\Bioamp_Data\Data\Jaw_Clench\Sandra_Jaw_Target_10000_Input.csv"ing EEG data is in the first column
        label = epoch.iloc[0, 1]  # Assuming the target is in the second column
        epochs.append(eeg_epoch)
        labels.append(label)

# Convert to NumPy arrays
epochs = np.array(epochs)
labels = np.array(labels)

# Perform label encoding for binary classification (REST, JAW)
label_encoder = LabelEncoder()
labels_encoded = label_encoder.fit_transform(labels)

# Normalize EEG data
scaler = MinMaxScaler(feature_range=(-1,1))
epochs = scaler.fit_transform(epochs)

Reshape data to match the input shape
epochs = epochs.reshape(epochs.shape[0], epochs.shape[1], 1)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(epochs, labels_encoded, test_size=0.3, random_state=42)

# Build the 1D CNN model
model = Sequential()
model.add(Conv1D(filters=5,kernel_size=3,strides=1))#1
model.add(BatchNormalization())
model.add(LeakyReLU())
model.add(MaxPool1D(pool_size=2,strides=2))#2
model.add(Conv1D(filters=5,kernel_size=3,strides=1))#3
model.add(LeakyReLU())
model.add(MaxPool1D(pool_size=2,strides=2))#4
model.add(Dropout(0.5))
model.add(Conv1D(filters=5,kernel_size=3,strides=1))#5
model.add(LeakyReLU())
model.add(AveragePooling1D(pool_size=2,strides=2))#6
model.add(Dropout(0.5))
model.add(Conv1D(filters=5,kernel_size=3,strides=1))#7
model.add(LeakyReLU())
model.add(AveragePooling1D(pool_size=2,strides=2))#8
model.add(Conv1D(filters=5,kernel_size=3,strides=1))#9
model.add(LeakyReLU())
model.add(GlobalAveragePooling1D())#10
model.add(Dense(3,activation='softmax'))#11
# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32 ,validation_data=(X_test, y_test))
# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Accuracy: {accuracy * 100:.2f}%')


Epoch 1/10


ValueError: in user code:

    File "C:\Users\HP\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py", line 1338, in train_function  *
        return step_function(self, iterator)
    File "C:\Users\HP\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py", line 1322, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\HP\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py", line 1303, in run_step  **
        outputs = model.train_step(data)
    File "C:\Users\HP\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py", line 1080, in train_step
        y_pred = self(x, training=True)
    File "C:\Users\HP\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "C:\Users

In [8]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder, MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPool1D, Dense, Flatten, BatchNormalization, LeakyReLU, Dropout, AveragePooling1D, GlobalAveragePooling1D
from tensorflow.keras.utils import to_categorical

# Load EEG data from CSV
eeg_data = pd.read_csv(r"D:\Joel\Bioamp_Data\Data\Jaw_Clench\2_mins_Alpha\2_min_Alpha_Input.csv")

# Define the number of rows in each epoch (4 seconds)
epoch_length = 25000

# Initialize lists to store epochs and labels
epochs = []
labels = []

# Extract EEG data and labels
for i in range(0, len(eeg_data), epoch_length):
    epoch = eeg_data.iloc[i:i + epoch_length]
    if len(epoch) == epoch_length:
        eeg_epoch = epoch.iloc[:, 0].values  # Assuming EEG data is in the first column
        label = epoch.iloc[0, 1]  # Assuming the target is in the second column
        epochs.append(eeg_epoch)
        labels.append(label)

# Convert to NumPy arrays
epochs = np.array(epochs)
labels = np.array(labels)

# Perform label encoding for multi-class classification
label_encoder = LabelEncoder()
labels_encoded = label_encoder.fit_transform(labels)
labels_one_hot = to_categorical(labels_encoded, num_classes=3)

# Normalize EEG data
scaler = MinMaxScaler(feature_range=(-1, 1))
epochs = scaler.fit_transform(epochs)

# Reshape data to match the input shape
epochs = epochs.reshape(epochs.shape[0], epochs.shape[1], 1)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(epochs, labels_one_hot, test_size=0.3, random_state=42)

# Build the 1D CNN model
model = Sequential()
model.add(Conv1D(filters=5, kernel_size=3, strides=1, input_shape=(epochs.shape[1], 1)))  # 1
model.add(BatchNormalization())
model.add(LeakyReLU())
model.add(MaxPool1D(pool_size=2, strides=2))  # 2
model.add(Conv1D(filters=5, kernel_size=3, strides=1))  # 3
model.add(LeakyReLU())
model.add(MaxPool1D(pool_size=2, strides=2))  # 4
model.add(Dropout(0.5))
model.add(Conv1D(filters=5, kernel_size=3, strides=1))  # 5
model.add(LeakyReLU())
model.add(AveragePooling1D(pool_size=2, strides=2))  # 6
model.add(Dropout(0.5))
model.add(Conv1D(filters=5, kernel_size=3, strides=1))  # 7
model.add(LeakyReLU())
model.add(AveragePooling1D(pool_size=2, strides=2))  # 8
model.add(Conv1D(filters=5, kernel_size=3, strides=1))  # 9
model.add(LeakyReLU())
model.add(GlobalAveragePooling1D())  # 10
model.add(Dense(3, activation='softmax'))  # 11

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Accuracy: {accuracy * 100:.2f}%')


Epoch 1/10
8/8 [==============================] - 5s 313ms/step - loss: nan - accuracy: 0.2311 - val_loss: nan - val_accuracy: 0.2621
Epoch 2/10
8/8 [==============================] - 2s 239ms/step - loss: nan - accuracy: 0.2311 - val_loss: nan - val_accuracy: 0.2621
Epoch 3/10
8/8 [==============================] - 2s 243ms/step - loss: nan - accuracy: 0.2311 - val_loss: nan - val_accuracy: 0.2621
Epoch 4/10
8/8 [==============================] - 2s 241ms/step - loss: nan - accuracy: 0.2311 - val_loss: nan - val_accuracy: 0.2621
Epoch 5/10
8/8 [==============================] - 2s 236ms/step - loss: nan - accuracy: 0.2311 - val_loss: nan - val_accuracy: 0.2621
Epoch 6/10
8/8 [==============================] - 2s 252ms/step - loss: nan - accuracy: 0.2311 - val_loss: nan - val_accuracy: 0.2621
Epoch 7/10
8/8 [==============================] - 2s 264ms/step - loss: nan - accuracy: 0.2311 - val_loss: nan - val_accuracy: 0.2621
Epoch 8/10
8/8 [==============================] - 2s 248ms/ste

In [11]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.utils import to_categorical

# Load EEG data from CSV
eeg_data = pd.read_csv(r'D:\Joel\Bioamp_Data\Data\Jaw_Clench\2_mins_Alpha\2_mins_Alpha_Blink\2_min_Alpha_Blink_Input.csv')

# Define the number of rows in each epoch (4 seconds)
epoch_length = 50000

# Initialize lists to store epochs and labels
epochs = []
labels = []

# Extract EEG data and labels
for i in range(0, len(eeg_data), epoch_length):
    epoch = eeg_data.iloc[i:i + epoch_length]
    if len(epoch) == epoch_length:
        eeg_epoch = epoch.iloc[:, 0].values  # Assuming EEG data is in the first column
        label = epoch.iloc[0, 1]  # Assuming the target is in the second column
        epochs.append(eeg_epoch)
        labels.append(label)

# Convert to NumPy arrays
epochs = np.array(epochs)
labels = np.array(labels)

# Perform label encoding for multi-class classification
label_encoder = LabelEncoder()
labels_encoded = label_encoder.fit_transform(labels)
labels_one_hot = to_categorical(labels_encoded, num_classes=3)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(epochs, labels_one_hot, test_size=0.3, random_state=42)

# Reshape data to match the input shape
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

# Define EEGNet architecture
model = keras.Sequential()
model.add(layers.Conv1D(8, kernel_size=64, input_shape=(X_train.shape[1], 1), padding='same'))
model.add(layers.BatchNormalization())
model.add(layers.DepthwiseConv1D(kernel_size=64, depth_multiplier=2, depthwise_initializer='he_normal', use_bias=False, padding='valid'))
model.add(layers.BatchNormalization())
model.add(layers.DepthwiseConv1D(kernel_size=16, depth_multiplier=2, depthwise_initializer='he_normal', use_bias=False, padding='same'))
model.add(layers.BatchNormalization())
model.add(layers.Activation('elu'))
model.add(layers.AveragePooling1D(pool_size=4))
model.add(layers.Dropout(0.5))
model.add(layers.SeparableConv1D(16, kernel_size=16, activation='elu', depthwise_initializer='he_normal', use_bias=False, padding='same'))
model.add(layers.BatchNormalization())
model.add(layers.SeparableConv1D(16, kernel_size=16, activation='elu', depthwise_initializer='he_normal', use_bias=False, padding='same'))
model.add(layers.BatchNormalization())
model.add(layers.AveragePooling1D(pool_size=4))
model.add(layers.Dropout(0.5))
model.add(layers.Flatten())
model.add(layers.Dense(3, activation='softmax'))  # Three-class classification

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=40, batch_size=32, validation_split=0.1)

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Loss: {loss}, Test Accuracy: {accuracy}")


Epoch 1/40
2/2 [==============================] - 10s 3s/step - loss: nan - accuracy: 0.4808 - val_loss: nan - val_accuracy: 0.3333
Epoch 2/40
2/2 [==============================] - 5s 2s/step - loss: nan - accuracy: 0.4808 - val_loss: nan - val_accuracy: 0.3333
Epoch 3/40
2/2 [==============================] - 6s 2s/step - loss: nan - accuracy: 0.4808 - val_loss: nan - val_accuracy: 0.3333
Epoch 4/40
2/2 [==============================] - 6s 2s/step - loss: nan - accuracy: 0.4808 - val_loss: nan - val_accuracy: 0.3333
Epoch 5/40
2/2 [==============================] - 5s 2s/step - loss: nan - accuracy: 0.4808 - val_loss: nan - val_accuracy: 0.3333
Epoch 6/40
2/2 [==============================] - 6s 2s/step - loss: nan - accuracy: 0.4808 - val_loss: nan - val_accuracy: 0.3333
Epoch 7/40
2/2 [==============================] - 6s 2s/step - loss: nan - accuracy: 0.4808 - val_loss: nan - val_accuracy: 0.3333
Epoch 8/40
2/2 [==============================] - 6s 2s/step - loss: nan - accurac

In [21]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.utils import to_categorical

# Step 1: Load the CSV file with no headers
file_path = r'D:\Joel\Bioamp_Data\Data\Jaw_Clench\2_mins_Alpha\2_mins_Alpha_Blink\2_min_Alpha_Blink_Input.csv'
df = pd.read_csv(file_path, header=None)

# Step 2: Extract Data and Targets
data = df.iloc[:, 0].values
targets = df.iloc[:, 1].values

# Step 3: Preprocess Data (Example: Normalize the data)
# You may need to perform additional preprocessing based on your requirements
data_normalized = (data - data.mean()) / data.std()

# Step 4: Label Encoding for Targets
label_encoder = LabelEncoder()
targets_encoded = label_encoder.fit_transform(targets)

# Step 5: Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(data_normalized, targets_encoded, test_size=0.3, random_state=42)

X_train_reshaped = X_train.reshape(X_train.shape[0],  1)  # Assuming X_train is 1D
X_test_reshaped = X_test.reshape(X_test.shape[0],  1)  # Assuming X_test is 1D
# Step 7: Build EEGNet Model
model = keras.Sequential()

model.add(layers.Conv1D(8, kernel_size=64, input_shape=(X_train_reshaped.shape[1], 1), padding='same'))
model.add(layers.BatchNormalization())
model.add(layers.DepthwiseConv1D(kernel_size=64, depth_multiplier=2, depthwise_initializer='he_normal', use_bias=False, padding='valid'))
model.add(layers.BatchNormalization())
model.add(layers.DepthwiseConv1D(kernel_size=16, depth_multiplier=2, depthwise_initializer='he_normal', use_bias=False, padding='same'))
model.add(layers.BatchNormalization())
model.add(layers.Activation('elu'))
model.add(layers.AveragePooling1D(pool_size=4))
model.add(layers.Dropout(0.5))
model.add(layers.SeparableConv1D(16, kernel_size=16, activation='elu', depthwise_initializer='he_normal', use_bias=False, padding='same'))
model.add(layers.BatchNormalization())
model.add(layers.SeparableConv1D(16, kernel_size=16, activation='elu', depthwise_initializer='he_normal', use_bias=False, padding='same'))
model.add(layers.BatchNormalization())
model.add(layers.AveragePooling1D(pool_size=4))
model.add(layers.Dropout(0.5))
model.add(layers.Flatten())
model.add(layers.Dense(3, activation='softmax'))  # Three-class classification

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train_reshaped, y_train, epochs=40, batch_size=32, validation_split=0.1)

# Evaluate the model
loss, accuracy = model.evaluate(X_test_reshaped, y_test)
print(f"Test Loss: {loss}, Test Accuracy: {accuracy}")


ValueError: Exception encountered when calling layer "depthwise_conv1d_7" (type DepthwiseConv1D).

Negative dimension size caused by subtracting 64 from 1 for '{{node depthwise_conv1d_7/depthwise}} = DepthwiseConv2dNative[T=DT_FLOAT, data_format="NHWC", dilations=[1, 1, 1, 1], explicit_paddings=[], padding="VALID", strides=[1, 1, 1, 1]](depthwise_conv1d_7/ExpandDims, depthwise_conv1d_7/ExpandDims_1)' with input shapes: [?,1,1,8], [1,64,8,2].

Call arguments received by layer "depthwise_conv1d_7" (type DepthwiseConv1D):
  • inputs=tf.Tensor(shape=(None, 1, 8), dtype=float32)

In [24]:
X_train.shape


(2936007,)

In [25]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, LSTM, Dense, Flatten, Dropout, Reshape
from tensorflow.keras.utils import to_categorical

# Load EEG data from CSV
eeg_data = pd.read_csv(r'D:\Joel\Bioamp_Data\Data\Jaw_Clench\2_mins_Alpha\2_mins_Alpha_Blink\2_min_Alpha_Blink_Input.csv')

# Define the number of rows in each epoch (4 seconds)
epoch_length = 50000

# Initialize lists to store epochs and labels
epochs = []
labels = []

# Extract EEG data and labels
for i in range(0, len(eeg_data), epoch_length):
    epoch = eeg_data.iloc[i:i + epoch_length]
    if len(epoch) == epoch_length:
        eeg_epoch = epoch.iloc[:, 0].values  # Assuming EEG data is in the first column
        label = epoch.iloc[0, 1]  # Assuming the target is in the second column
        epochs.append(eeg_epoch)
        labels.append(label)

# Convert to NumPy arrays
epochs = np.array(epochs)
labels = np.array(labels)

# Perform label encoding for multi-class classification
label_encoder = LabelEncoder()
labels_encoded = label_encoder.fit_transform(labels)
labels_one_hot = to_categorical(labels_encoded, num_classes=np.max(labels_encoded) + 1)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(epochs, labels_one_hot, test_size=0.3, random_state=42)

# Reshape the input data for a 1D CNN
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

# Define the model
model = Sequential()

# Convolutional layers for spatial feature extraction
model.add(Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=(X_train.shape[1], 1)))
model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))

# Reshape to prepare for LSTM layers
model.add(Reshape((-1, 64)))

# LSTM layers for capturing temporal dependencies
model.add(LSTM(units=50, return_sequences=True))
model.add(LSTM(units=50))

# Flatten LSTM output
model.add(Flatten())

# Dense layers for classification
model.add(Dense(units=128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(units=np.max(labels_encoded) + 1, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)

# Evaluate the model on the test set
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Loss: {loss}, Test Accuracy: {accuracy}')


Epoch 1/10
2/2 [==============================] - 105s 33s/step - loss: nan - accuracy: 0.5217 - val_loss: nan - val_accuracy: 0.2500
Epoch 2/10
2/2 [==============================] - 44s 21s/step - loss: nan - accuracy: 0.5217 - val_loss: nan - val_accuracy: 0.2500
Epoch 3/10
2/2 [==============================] - 46s 23s/step - loss: nan - accuracy: 0.5217 - val_loss: nan - val_accuracy: 0.2500
Epoch 4/10
2/2 [==============================] - 46s 23s/step - loss: nan - accuracy: 0.5217 - val_loss: nan - val_accuracy: 0.2500
Epoch 5/10
2/2 [==============================] - 46s 22s/step - loss: nan - accuracy: 0.5217 - val_loss: nan - val_accuracy: 0.2500
Epoch 6/10
2/2 [==============================] - 48s 23s/step - loss: nan - accuracy: 0.5217 - val_loss: nan - val_accuracy: 0.2500
Epoch 7/10
2/2 [==============================] - 49s 23s/step - loss: nan - accuracy: 0.5217 - val_loss: nan - val_accuracy: 0.2500
Epoch 8/10
2/2 [==============================] - 49s 23s/step - los

In [26]:
# Evaluate the model on the test set
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Loss: {loss}, Test Accuracy: {accuracy}')

# Predict classes for each sample in the test set
y_pred = model.predict(X_test)
predicted_classes = np.argmax(y_pred, axis=1)

# Convert one-hot encoded labels back to original labels
true_classes = np.argmax(y_test, axis=1)

# Calculate accuracy for each class
class_accuracy = {}
for class_label in np.unique(true_classes):
    class_indices = np.where(true_classes == class_label)[0]
    correct_predictions = np.sum(predicted_classes[class_indices] == true_classes[class_indices])
    total_samples = len(class_indices)
    class_accuracy[class_label] = correct_predictions / total_samples

# Print accuracy for each class
print("Accuracy for each class:")
for class_label, acc in class_accuracy.items():
    print(f"Class {class_label}: {acc * 100:.2f}%")


1/1 [==============================] - 2s 2s/step - loss: nan - accuracy: 0.5200
Test Loss: nan, Test Accuracy: 0.5199999809265137
1/1 [==============================] - 3s 3s/step
Accuracy for each class:
Class 0: 100.00%
Class 1: 0.00%


In [7]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from mne.decoding import CSP
from tensorflow import keras
from tensorflow.keras import layers

# Load EEG data from CSV
eeg_data = pd.read_csv(r"D:\Joel\Bioamp_Data\Data\Jaw_Clench\2_mins_Alpha\JAW\Joel_2min_alpha.csv")

# Define the number of rows in each epoch (4 seconds)
epoch_length = 50000

# Initialize lists to store epochs and labels
epochs = []
labels = []

# Extract EEG data and labels
for i in range(0, len(eeg_data), epoch_length):
    epoch = eeg_data.iloc[i:i + epoch_length]
    if len(epoch) == epoch_length:
        eeg_epoch = epoch.iloc[:, 0].values  # Assuming EEG data is in the first column
        label = epoch.iloc[0, 1]  # Assuming the target is in the second column
        epochs.append(eeg_epoch)
        labels.append(label)

# Convert to NumPy arrays
epochs = np.array(epochs)
labels = np.array(labels)

# Perform label encoding for three-class classification
label_encoder = LabelEncoder()
labels_encoded = label_encoder.fit_transform(labels)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(epochs, labels_encoded, test_size=0.3, random_state=42)

# Reshape data to match the input shape
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)
y_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
y_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

# Check dimensions of X_train
print("X_train shape:", X_train.squeeze().T.shape,y_train.shape)

# Check length of y_train
print("Length of y_train:", len(y_train))

# Apply CSP
csp = CSP(n_components=4, reg=None, log=True, norm_trace=False)
X_train_csp = csp.fit_transform(X_train.squeeze().T, y_train)
X_test_csp = csp.transform(X_test.squeeze().T)

# Define EEGNet architecture with CSP
model = keras.Sequential()
model.add(layers.Flatten(input_shape=(X_train_csp.shape[1],)))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(32, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(3, activation='softmax'))  # Three-class classification

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train_csp, y_train, epochs=40, batch_size=32, validation_split=0.1)

# Evaluate the model
loss, accuracy = model.evaluate(X_test_csp, y_test)
print(f"Test Loss: {loss}, Test Accuracy: {accuracy}")


X_train shape: (50000, 14) (14, 50000, 1)
Length of y_train: 14


ValueError: X and y must have the same length.

In [8]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from mne.decoding import CSP
from tensorflow import keras
from tensorflow.keras import layers

# Load EEG data from CSV
eeg_data = pd.read_csv(r"D:\Joel\Bioamp_Data\Data\Jaw_Clench\2_mins_Alpha\JAW\Joel_2min_alpha.csv")

# Define the number of rows in each epoch (4 seconds)
epoch_length = 25000

# Initialize lists to store epochs and labels
epochs = []
labels = []

# Extract EEG data and labels
for i in range(0, len(eeg_data), epoch_length):
    epoch = eeg_data.iloc[i:i + epoch_length]
    if len(epoch) == epoch_length:
        eeg_epoch = epoch.iloc[:, 0].values  # Assuming EEG data is in the first column
        label = epoch.iloc[0, 1]  # Assuming the target is in the second column
        epochs.append(eeg_epoch)
        labels.append(label)

# Convert to NumPy arrays
epochs = np.array(epochs)
labels = np.array(labels)

# Perform label encoding for three-class classification
label_encoder = LabelEncoder()
labels_encoded = label_encoder.fit_transform(labels)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(epochs, labels_encoded, test_size=0.3, random_state=42)

# Reshape data to match the input shape
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

# Apply CSP
csp = CSP(n_components=4, reg=None, log=True, norm_trace=False)
X_train_csp = csp.fit_transform(X_train.squeeze().T, y_train)
X_test_csp = csp.transform(X_test.squeeze().T)

# Define EEGNet architecture with CSP
model = keras.Sequential()
model.add(layers.Flatten(input_shape=(X_train_csp.shape[1],)))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(32, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(3, activation='softmax'))  # Three-class classification

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train_csp, y_train, epochs=40, batch_size=32, validation_split=0.1)

# Evaluate the model
loss, accuracy = model.evaluate(X_test_csp, y_test)
print(f"Test Loss: {loss}, Test Accuracy: {accuracy}")


ValueError: X and y must have the same length.